In [1]:
from glob import glob

import os
import pandas as pd
import numpy as np
import json
from pathlib import Path
import torch
import pickle

In [2]:
order = ["DeepFool", "BIM", "CW", "Low_CW", "FAB", "FGSM", "PGD", "Low_PGD1", "Low_PGD2"]

In [15]:
dataname = 'CIFAR100'
modelname = 'vgg19'

# Adversarial Attack

In [16]:
del_keys = ['clean acc','noise acc','clean dwt acc']

attack_results = glob(f'./results/{dataname}/saved_adv_samples/{modelname}/*/results.json')
adv_df = pd.DataFrame()

for p in attack_results:
    r = json.load(open(p,'r'))
    
    for k in del_keys:
        del r[k]
    
    r = pd.Series(
        r,
        name = os.path.basename(os.path.dirname(p))
    )
    adv_df = pd.concat([adv_df, r], axis=1)
    
adv_df = ((adv_df.T.loc[order]) * 100).round(2)

adv_df.columns = ['Adv Acc(%)','Adv Acc(%) DWT']

## Successed Images

In [17]:
succ_list = glob(f'./results/{dataname}/saved_adv_samples/{modelname}/*/successed_images.pkl')

In [18]:
for p in succ_list:
    with open(p,'rb') as f:
        nb_imgs = pickle.load(f)['clean'].shape[0]
        adv_method = os.path.basename(os.path.dirname(p))
        adv_df.loc[adv_method,'# Success Images'] = nb_imgs

adv_df['# Success Images'] = adv_df['# Success Images'].astype(int)

In [19]:
print(adv_df.to_markdown())

|          |   Adv Acc(%) |   Adv Acc(%) DWT |   # Success Images |
|:---------|-------------:|-----------------:|-------------------:|
| DeepFool |         2.9  |            57.6  |               2961 |
| BIM      |         1.72 |            44.59 |               2950 |
| CW       |         9.35 |            52.97 |               2245 |
| Low_CW   |        30.82 |            59.3  |               1187 |
| FAB      |         5.31 |            59.14 |               2865 |
| FGSM     |        16.79 |            35.4  |               1826 |
| PGD      |         1.32 |            45.68 |               2943 |
| Low_PGD1 |        29.61 |            58.47 |               1190 |
| Low_PGD2 |         9.72 |            56.72 |               2325 |


# Detection

## Known Attack

In [20]:
del_keys = ['TNR','AUIN','AUOUT']

known_results = glob(f'./results/{dataname}/known_attack_results/{modelname}/*/result.json')
known_df = pd.DataFrame()

for p in known_results:
    r = json.load(open(p,'r'))['test']

    for k in del_keys:
        del r[k]

    r = pd.Series(
        r,
        name = os.path.basename(os.path.dirname(p))
    )
    
    known_df = pd.concat([known_df, r], axis=1)

known_df = ((known_df.T.loc[order]) * 100).round(2)

known_df.columns = ['AUROC(%)','Detection Acc(%)']

## Train and Test images

In [21]:
train_det = glob(f'./results/{dataname}/known_attack_results/{modelname}/*/train.pt')
dev_det = glob(f'./results/{dataname}/known_attack_results/{modelname}/*/dev.pt')
test_det = glob(f'./results/{dataname}/known_attack_results/{modelname}/*/test.pt')

In [22]:
for train_p, dev_p, test_p in zip(train_det, dev_det, test_det):
    nb_train = torch.load(train_p)['logits'].size(0)
    nb_dev = torch.load(dev_p)['logits'].size(0)    
    nb_test = torch.load(test_p)['logits'].size(0)
    
    adv_method = os.path.basename(os.path.dirname(train_p))
    
    known_df.loc[adv_method, '#(train, dev, test)'] = f"({nb_train}, {nb_dev}, {nb_test})"

In [23]:
print(known_df.to_markdown())

|          |   AUROC(%) |   Detection Acc(%) | #(train, dev, test)   |
|:---------|-----------:|-------------------:|:----------------------|
| DeepFool |      80.4  |              73.61 | (1893, 315, 949)      |
| BIM      |      86.14 |              78.24 | (1879, 312, 943)      |
| CW       |      72.53 |              68.66 | (1461, 243, 734)      |
| Low_CW   |      59.76 |              59.38 | (779, 129, 392)       |
| FAB      |      76.26 |              70.87 | (1836, 305, 920)      |
| FGSM     |      63.77 |              62.82 | (1247, 206, 627)      |
| PGD      |      87.85 |              81.94 | (1872, 310, 940)      |
| Low_PGD1 |      55.68 |              58.55 | (782, 129, 395)       |
| Low_PGD2 |      66.83 |              67.21 | (1483, 246, 745)      |


## Transfer Attack

In [24]:
transfer_df = pd.read_csv(f'./results/{dataname}/transfer_attack_results/{modelname}/transfer_results.csv',index_col=0)
print(transfer_df.loc[order, order].to_markdown())

|          |   DeepFool |   BIM |    CW |   Low_CW |   FAB |   FGSM |   PGD |   Low_PGD1 |   Low_PGD2 |
|:---------|-----------:|------:|------:|---------:|------:|-------:|------:|-----------:|-----------:|
| DeepFool |      80.4  | 52.95 | 71.77 |    81.31 | 82.27 |  68.61 | 51.34 |      72.65 |      70.04 |
| BIM      |      46.63 | 86.14 | 57.88 |    53.55 | 41.39 |  61.81 | 90.2  |      56.72 |      73.45 |
| CW       |      75.26 | 72.3  | 72.53 |    69.18 | 72.6  |  67.45 | 65.02 |      66.54 |      71.74 |
| Low_CW   |      83.81 | 52.64 | 83.21 |    59.76 | 78.18 |  60.5  | 52.16 |      62.42 |      72.59 |
| FAB      |      84.15 | 53.52 | 72.35 |    74.6  | 76.26 |  58.67 | 54.37 |      65.21 |      66.54 |
| FGSM     |      72.34 | 61.55 | 71.93 |    67.63 | 71.73 |  63.77 | 62.16 |      63.21 |      64.98 |
| PGD      |      44.98 | 89.12 | 51.08 |    45.1  | 40.51 |  56.23 | 87.85 |      52.43 |      66.13 |
| Low_PGD1 |      76.1  | 57.68 | 73.99 |    75.12 | 77.17 |  61